In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from typing import List,Dict
from nltk.stem import SnowballStemmer as sbs
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt


In [2]:
emails = pd.read_csv("emails-small.csv")

In [3]:
#emails = pd.read_csv("emails.csv")
#df = emails.iloc[0:1000,:]
#df.to_csv("emails-small.csv")

In [4]:
#nltk.download("wordnet")
#nltk.download("stopwords")

In [5]:
def create_variables(data):
    """
    This function will create 5 lists. Taking the data from
    the emails["messages"], we can found the "from" the "to" the "date"
    the "subject" of the email for every mail
    """
    Body_list = []
    From_list = []
    To_list = []
    Date_list = []
    Subject_list = []

    for data_message in data:

        email = data_message.split("\n")
        end = False
        message = []

        for part in email:
            if end == False:
                if part.find(":"):
                    if part.startswith("Date:"):
                        Date = part.split(": ")[1]
                    if part.startswith("From:"):
                        From = part.split(": ")[1]
                    if part.startswith("To:"):
                        To = part.split(": ")[1]
                    if part.startswith("Subject:"):
                        Subject = part.split(": ")[1]
                    if part.startswith("X-FileName:"):
                        end = True
            else:
                message.append(part)

        Body = "\n".join(message) #put in one string the body of the email


        From_list.append(From)
        To_list.append(To)
        Subject_list.append(Subject)
        Date_list.append(Date)
        Body_list.append(Body)

    return From_list,To_list,Date_list, Subject_list, Body_list
                


In [6]:
From,To,Date, Subject, Body = create_variables(emails["message"])

In [7]:
def create_columns(From,To,Date,Subject,Body,emails):
    """
    This function will create 5 news columns in the dataframe
    named emails
    """
    emails["From"] = From
    emails["To"] = To
    emails["Date"] = Date
    emails["Subject"] = Subject
    emails["Body"] = Body
    
    return emails

In [8]:
emails = create_columns(From,To,Date,Subject,Body,emails)

In [9]:
emails

,Unnamed: 0,file,message,From,To,Date,Subject,Body
0,0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,phillip.allen@enron.com,tim.belden@enron.com,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,\nHere is our forecast\n\n
1,1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,phillip.allen@enron.com,john.lavorato@enron.com,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,\nTraveling to have a business meeting takes t...
2,2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,phillip.allen@enron.com,leah.arsdall@enron.com,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re,\ntest successful. way to go!!!
3,3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,phillip.allen@enron.com,randall.gay@enron.com,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",,"\nRandy,\n\n Can you send me a schedule of the..."
4,4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,phillip.allen@enron.com,greg.piper@enron.com,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Re,\nLet's shoot for Tuesday at 11:45.
...,...,...,...,...,...,...,...,...
995,995,allen-p/all_documents/458.,Message-ID: <20430828.1075855696096.JavaMail.e...,phillip.allen@enron.com,jacquestc@aol.com,"Mon, 19 Mar 2001 01:36:00 -0800 (PST)",,"\nJacques,\n\nStill trying to close the loop o..."
996,996,allen-p/all_documents/459.,Message-ID: <18425275.1075855696118.JavaMail.e...,phillip.allen@enron.com,llewter@austin.rr.com,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)",Re,"\nLarrry,\n\nI realize you are disappointed ab..."
997,997,allen-p/all_documents/46.,Message-ID: <24036204.1075855666506.JavaMail.e...,phillip.allen@enron.com,pallen70@hotmail.com,"Wed, 6 Dec 2000 08:04:00 -0800 (PST)",,\n---------------------- Forwarded by Phillip ...
998,998,allen-p/all_documents/460.,Message-ID: <33307764.1075855696139.JavaMail.e...,phillip.allen@enron.com,jacquestc@aol.com,"Fri, 16 Mar 2001 04:28:00 -0800 (PST)",,"\nJacques,\n\nI think we reached an agreement ..."


In [10]:
test = emails["Body"][12]
test

"\n---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/09/2000 \n02:16 PM ---------------------------\n\n\nRichard Burchfield\n10/06/2000 06:59 AM\nTo: Phillip K Allen/HOU/ECT@ECT\ncc: Beth Perlman/HOU/ECT@ECT \nSubject: Consolidated positions: Issues & To Do list\n\nPhillip,\n Below is the issues & to do list as we go forward with documenting the \nrequirements for consolidated physical/financial positions and transport \ntrade capture. What we need to focus on is the first bullet in Allan's list; \nthe need for a single set of requirements. Although the meeting with Keith, \non Wednesday,  was informative the solution of creating a infinitely dynamic \nconsolidated position screen, will be extremely difficult and time \nconsuming.  Throughout the meeting on Wednesday, Keith alluded to the \ninability to get consensus amongst the traders on the presentation of the \nconsolidated position, so the solution was to make it so that a trader can \narrange the position screen t

In [11]:
def get_words_list(body:str) -> List[str]:
    """
    This function is kinda of manual tokenization
    """
    pattern = "\\S+"
    my_list = re.findall(pattern,body)
    my_string = " ".join(my_list)
    pattern = "\\w+"
    my_list = re.findall(pattern,my_string)
    my_string = " ".join(my_list)
    pattern = "\\D+"
    my_list = re.findall(pattern,my_string)
    return my_list

In [12]:
body = get_words_list(test)

In [43]:
body

['Forwarded by Phillip K Allen HOU ECT on ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' PM Richard Burchfield ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' AM To Phillip K Allen HOU ECT ECT cc Beth Perlman HOU ECT ECT Subject Consolidated positions Issues To Do list Phillip Below is the issues to do list as we go forward with documenting the requirements for consolidated physical financial positions and transport trade capture What we need to focus on is the first bullet in Allan s list the need for a single set of requirements Although the meeting with Keith on Wednesday was informative the solution of creating a infinitely dynamic consolidated position screen will be extremely difficult and time consuming Throughout the meeting on Wednesday Keith alluded to the inability to get consensus amongst the traders on the presentation of the consolidated position so the solution was to make it so that a trader can arrange the position screen to their liking much like Excel What needs to happen on Monday from ',
 ' ',
 ' is

In [44]:
def stemmed_word(body:List[str]) -> List[str]:
    """
    This function replace the list of word by the list of 
    root of the word
    """
    #choose english words
    stemmer = sbs("english")
    body_stemmed = []
    for word in body:
        body_stemmed.append(stemmer.stem(word))
    
    return body_stemmed

In [45]:
stemmed_body = stemmed_word(body)

In [46]:
def clean_word(body:List[str]) -> List[str]:
    """
    clean the list of word in the body of the email. 
    Delete every word contain in the stop word list.
    """
    #create stop word list
    stop_words = stopwords.words("english")
    body_cleaned = []
    for word in body:
        if word.lower() not in stop_words:
            body_cleaned.append(word)
    
    print(stop_words)
    return list(set(body_cleaned)) #delete duplicate

In [47]:
body_cleaned = clean_word(stemmed_body)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [48]:
body_cleaned

[' ',
 ' am allan severude ',
 'forwarded by phillip k allen hou ect on ',
 ' am to phillip k allen hou ect ect cc beth perlman hou ect ect subject consolidated positions issues to do list phillip below is the issues to do list as we go forward with documenting the requirements for consolidated physical financial positions and transport trade capture what we need to focus on is the first bullet in allan s list the need for a single set of requirements although the meeting with keith on wednesday was informative the solution of creating a infinitely dynamic consolidated position screen will be extremely difficult and time consuming throughout the meeting on wednesday keith alluded to the inability to get consensus amongst the traders on the presentation of the consolidated position so the solution was to make it so that a trader can arrange the position screen to their liking much like excel what needs to happen on monday from ',
 ' pm to richard burchfield hou ect ect cc peggy alix hou

In [49]:
def create_document_term_matrix(body):
    cv = CountVectorizer()
    X = cv.fit(body)
    my_columns = X.get_feature_names_out()
    X = cv.transform(body)
    my_values = X.toarray()
    df = pd.DataFrame(my_values,columns=my_columns)
    return df

In [50]:
df = create_document_term_matrix(body_cleaned)

In [51]:
df

,ability,above,accomplish,across,ad,add,addition,affect,aggregation,alix,...,was,we,wednesday,what,where,which,will,with,work,would
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,0,0,1,2,0,0
4,8,2,1,1,1,1,1,1,1,1,...,0,7,0,1,1,6,0,5,2,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,2,0,0,0


In [59]:
def get_top_10_dt_matrix(df):
    return list(df.sum().sort_values(ascending=False).head(10).index.values)

In [60]:
get_top_10_dt_matrix(df)

['the', 'to', 'of', 'position', 'for', 'in', 'ect', 'and', 'on', 'we']

In [53]:
emails

,Unnamed: 0,file,message,From,To,Date,Subject,Body
0,0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,phillip.allen@enron.com,tim.belden@enron.com,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,\nHere is our forecast\n\n
1,1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,phillip.allen@enron.com,john.lavorato@enron.com,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,\nTraveling to have a business meeting takes t...
2,2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,phillip.allen@enron.com,leah.arsdall@enron.com,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re,\ntest successful. way to go!!!
3,3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,phillip.allen@enron.com,randall.gay@enron.com,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",,"\nRandy,\n\n Can you send me a schedule of the..."
4,4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,phillip.allen@enron.com,greg.piper@enron.com,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Re,\nLet's shoot for Tuesday at 11:45.
...,...,...,...,...,...,...,...,...
995,995,allen-p/all_documents/458.,Message-ID: <20430828.1075855696096.JavaMail.e...,phillip.allen@enron.com,jacquestc@aol.com,"Mon, 19 Mar 2001 01:36:00 -0800 (PST)",,"\nJacques,\n\nStill trying to close the loop o..."
996,996,allen-p/all_documents/459.,Message-ID: <18425275.1075855696118.JavaMail.e...,phillip.allen@enron.com,llewter@austin.rr.com,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)",Re,"\nLarrry,\n\nI realize you are disappointed ab..."
997,997,allen-p/all_documents/46.,Message-ID: <24036204.1075855666506.JavaMail.e...,phillip.allen@enron.com,pallen70@hotmail.com,"Wed, 6 Dec 2000 08:04:00 -0800 (PST)",,\n---------------------- Forwarded by Phillip ...
998,998,allen-p/all_documents/460.,Message-ID: <33307764.1075855696139.JavaMail.e...,phillip.allen@enron.com,jacquestc@aol.com,"Fri, 16 Mar 2001 04:28:00 -0800 (PST)",,"\nJacques,\n\nI think we reached an agreement ..."
